In [8]:
from fastai.structured import *
from fastai.column_data import *
import pandas as pd
np.set_printoptions(threshold=50, edgeitems=20)

PATH = 'orig_pdb_fixed/'

In [9]:
from biopandas.pdb import PandasPdb

In [10]:
path = 'orig_pdb_fixed'

In [11]:
import glob

In [12]:
from sklearn import preprocessing

In [13]:
 all_files = glob.glob(os.path.join(PATH, "*")) 

In [14]:
all_files[:5]

['orig_pdb_fixed/2vtwA00.pdb',
 'orig_pdb_fixed/3mtuE00.pdb',
 'orig_pdb_fixed/2jzxA02.pdb',
 'orig_pdb_fixed/4i59A03.pdb',
 'orig_pdb_fixed/4fkcA01.pdb']

In [16]:
pl1 = PandasPdb().read_pdb(all_files[0])

In [18]:
scaler = preprocessing.StandardScaler()

In [19]:
pl1.df['ATOM'][['x_coord','y_coord','z_coord']]= scaler.fit_transform(pl1.df['ATOM'][['x_coord','y_coord','z_coord']])


In [20]:
f = all_files[0].split('/')[1]
ff = f.split('.')[0]
pl1.to_pdb(path=f'normal_pdb/{ff}.pdb', 
            records=['ATOM'], 
            gz=False, 
            append_newline=True)

In [28]:
pf = PandasPdb().read_pdb(feature_dict[mutual_files[0]])

In [30]:
pf.df['ATOM'].loc[(pf.df['ATOM'].atom_name == 'CA') , 'x_coord'] 

0       73.645
4       69.865
13      68.886
24      66.575
32      64.828
43      64.406
50      61.075
61      61.507
69      58.311
76      59.948
83      58.394
91      54.968
102     56.113
110     56.335
114     58.059
122     57.525
130     57.530
137     61.331
146     62.545
155     60.721
164     62.732
172     65.956
180     65.100
187     63.997
192     67.298
201     69.217
206     67.301
214     68.101
222     71.815
231     71.706
         ...  
1733    72.431
1738    75.591
1747    77.308
1753    76.065
1763    76.247
1772    78.156
1781    75.533
1788    72.402
1796    73.404
1800    71.491
1808    72.349
1817    70.696
1824    72.077
1828    72.330
1836    75.480
1844    76.479
1854    79.063
1863    76.591
1872    74.265
1880    75.037
1887    75.231
1894    72.200
1902    70.391
1908    71.511
1918    70.168
1926    66.605
1933    66.285
1939    67.315
1946    65.369
1954    63.169
Name: x_coord, Length: 259, dtype: float64

In [18]:
p1.df['ATOM'].loc[(p1.df['ATOM'].atom_name == 'CA') , 'x_coord'] 

1       73.645
5       69.865
14      68.886
25      66.575
33      64.828
44      64.406
51      61.075
62      61.507
70      58.311
77      59.948
84      58.394
92      54.968
103     56.113
111     56.335
115     58.059
123     57.525
131     57.530
138     61.331
147     62.545
156     60.721
165     62.732
173     65.956
181     65.100
188     63.997
193     67.298
202     69.217
207     67.301
215     68.101
223     71.815
232     71.706
         ...  
1734    72.431
1739    75.591
1748    77.308
1754    76.065
1764    76.247
1773    78.156
1782    75.533
1789    72.402
1797    73.404
1801    71.491
1809    72.349
1818    70.696
1825    72.077
1829    72.330
1837    75.480
1845    76.479
1855    79.063
1864    76.591
1873    74.265
1881    75.037
1888    75.231
1895    72.200
1903    70.391
1909    71.511
1919    70.168
1927    66.605
1934    66.285
1940    67.315
1947    65.369
1955    63.169
Name: x_coord, Length: 259, dtype: float64

In [31]:
p1.df['ATOM'].loc[(p1.df['ATOM'].atom_name == 'CA') , 'x_coord'] = pf.df['ATOM'].loc[(p1.df['ATOM'].atom_name == 'CA') , 'x_coord'] 

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [22]:

c = 0
for f in mutual_files:
    df_t = pd.read_pickle(pickle_dict[f])
    df=pd.read_pickle('merged_1m.pkl')
    cat_vars = ['atom_name', 'residue_name_x', 'chain_id_y',
       'element_symbol_y']
    contin_vars = ['x_coord', 'y_coord', 'z_coord','feature_x', 'feature_y', 'feature_z']
    for v in cat_vars: df[v] = df[v].astype('category').cat.as_ordered()
    for v in contin_vars:
        df[v] = df[v].fillna(0).astype('float32')
    y=df.drop(['atom_name', 'residue_name_x', 'chain_id_y','b_factor_y','element_symbol_y', 'feature_x', 'feature_y', 'feature_z'],axis=1)
    y=y.values
    train_cats(df)
    train_cats(df_t)
    df=df.drop(['x_coord','y_coord', 'z_coord'],axis=1)
    df_t=df_t.drop(['x_coord','y_coord', 'z_coord'],axis=1)
    df1, d, nas, mapper = proc_df(df, 'b_factor_y', do_scale=True)
    df_test, _, nas, mapper = proc_df(df_t, 'b_factor_y', do_scale=True,
                                  mapper=mapper, na_dict=nas)
    train_ratio = 0.9
    train_size = int(len(df1) * train_ratio); train_size
    val_idx = list(range(train_size, len(df1)))
    md = ColumnarModelData.from_data_frame(PATH, val_idx, df1, y.astype(np.float32), cat_flds=cat_vars, bs=128 ,test_df= df_test)
    cat_sz = [(c, len(df[c].cat.categories)+1) for c in cat_vars]
    emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]
    m = md.get_learner(emb_szs, 3,0.08, 3, [1000,500,500,500,500,500,500], [0.3,0.3])
    m.crit = F.mse_loss
    m.load('m_feature')
    pred_test=m.predict(True)
    xi = pred_test[:,0]
    yi = pred_test[:,1]
    zi = pred_test[:,2]
    
#     xi = np.empty((len(pred_test),1))
#     for i in range(len(pred_test)):
#         b=pred_test[i][0]
#         xi[i]=b
    
#     yi = np.empty((len(pred_test),1))
#     for j in range(len(pred_test)):
#         h=pred_test[i][1]
#         yi[j]=h
#     zi = np.empty((len(pred_test),1))
#     for k in range(len(pred_test)):
#         l=pred_test[i][2]
#         zi[k]=l
    xi = xi.reshape(len(pred_test))
    yi = yi.reshape(len(pred_test))
    zi = zi.reshape(len(pred_test))
    xi = pd.Series(xi)
    yi = pd.Series(yi)
    zi = pd.Series(zi)
    pdb = PandasPdb().read_pdb(pdb_dict[f])
    pdb.df['ATOM']['x_coord'] = xi
    pdb.df['ATOM']['y_coord'] = yi
    pdb.df['ATOM']['z_coord'] = zi
    
    pdb.to_pdb(path=f'test_set/new_pdb/{f}.pdb', 
            records=['ATOM'], 
            gz=False, 
            append_newline=True)
    print(c)
    c=c+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [18]:
pred_test[:10]

array([[43.89424, 59.68964, 16.65406],
       [42.84148, 60.19216, 15.79624],
       [43.08684, 61.58086, 15.15461],
       [43.24935, 61.71384, 14.17512],
       [41.58088, 59.55253, 15.92995],
       [41.32315, 57.84309, 15.2151 ],
       [40.88296, 56.90148, 15.65617],
       [42.62218, 57.51491, 14.16098],
       [42.96107, 62.65746, 15.98621],
       [42.69254, 63.90514, 15.59726]], dtype=float32)

In [21]:
pred_test[:,2]

array([16.65406, 15.79624, 15.15461, 14.17512, 15.92995, 15.2151 , 15.65617, 14.16098, 15.98621, 15.59726,
       15.37395, 15.97272, 16.25845, 16.56664, 17.40854, 15.49968, 14.7758 , 14.14398, 13.99648, 12.75414,
       ..., 14.18698, 15.21259, 13.4437 , 13.97407, 13.1443 , 12.55043, 13.23126, 13.25438, 13.43075,
       11.32205, 10.17307,  8.92626,  8.33644, 10.30026,  9.08449, 10.97833,  8.12679,  6.75556,  5.74731,
        5.1857 ], dtype=float32)